<a href="https://colab.research.google.com/github/amirkfard/Deep-Q-Leaning-Infrastructure-network-planning/blob/main/DQL_Sewer_Flushing_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from tqdm import tqdm,tnrange
# from pulp import *
from scipy.linalg import expm
# from ortools.linear_solver import pywraplp
# from ortools.sat.python import cp_model

import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import elitism

import os , time , copy
import pickle , json

# set the random seed:
RANDOM_SEED = 10
random.seed(RANDOM_SEED)

Criteria = "MC"
num_catchments = 7
num_years = 7
Ucost = 1.38
a = 0.1
budgetb = 10000
#budget = [budgetb for i in range(num_years)] #Budget Profile
#budget = [45000,50000,55000,60000,65000,70000,75000] #Budget Profile
Tol=0.15

dfCL = pd.read_pickle(r'/content/drive/MyDrive/Data/dfCL').round(2)
dfT = pd.read_pickle(r'/content/drive/MyDrive/Data/dfT').round(2)
dfT.rename(columns={'SHAPE_Length': 'Len'}, inplace=True)
dfT.rename(columns={'SHAPE_Length': 'Len'}, inplace=True)

dfCL = dfCL.iloc[0:num_catchments]
catchments = dfCL.index

dfCL = pd.DataFrame(dfCL)
dfCL.columns = ['Len']

dfT["Tran"] = ""
dfT["Init"] = ""
dfT["Tran"] = dfT["Tran"].astype(object)
dfT["Init"] = dfT["Tran"].astype(object)
dfT["Next"] = ""
dfT["Mean"] = ""
dfT["Next"] = dfT["Next"].astype(object)

dfCL["Tran"] = ""
dfCL["Init"] = ""
dfCL["Next"] = ""
dfCL["Mean"] = ""
dfCL["MeanLen"] = ""
dfCL["PPA"] = ""
dfCL["PPALen"] = ""
dfCL["TPR"] = ""
dfCL["Tran"] = dfCL["Tran"].astype(object)
dfCL["Init"] = dfCL["Init"].astype(object)
dfCL["Next"] = dfCL["Next"].astype(object)
dfCL["TPR"] = dfCL["TPR"].astype(object)


for i in range(len(dfT)):
    a = 1-dfT.Degradation_Rate[i]
    lambda1 = dfT.lambda1[i]
    lambda2 = dfT.lambda2[i]
    lambda3 = dfT.lambda3[i]
    lambda4 = dfT.lambda4[i]
    Q = np.array([[-lambda1, lambda1, 0, 0, 0],
                  [0, -lambda2, lambda2, 0, 0],
                  [0, 0, -lambda3, lambda3, 0],
                  [0, 0, 0, -lambda4, lambda4],
                  [0, 0, 0, 0, 0]])
    Q = Q * 1                ######################################  Degradation Rate Factor #####################
    dfT.Tran[i] = np.around(expm(Q), decimals=3)
    print(np.around(Q, decimals=3))
    print(dfT.Tran[i])
    if np.isnan(np.sum(dfT.Tran[i])):
        dfT.Tran[i]=np.array([[0, 0, 0, 0, 1],
                              [0, 0, 0, 0, 1],
                              [0, 0, 0, 0, 1],
                              [0, 0, 0, 0, 1],
                              [0, 0, 0, 0, 1]])


    
    dfT.Init[i]=np.array([0, 0, 0, 0, 0])
    if dfT.Current[i]==0:
        dfT.Init[i][0]=1
    else:
        dfT.Init[i][dfT.Current[i]-1]=1


        
for catchment in catchments:
    dfC = dfT.loc[dfT['Catchment_Group'] == catchment].copy()
    dfC['TranLen']=dfC.Tran*dfC.Len
    STranLen = sum(dfC['TranLen'])
    net_tran = STranLen/np.sum(STranLen,1)
    dfCL.Tran[catchment] = net_tran
   
    dfC['InitLen']=dfC.Init*dfC.Len
    SInitLen=sum(dfC['InitLen'])
    net_init = SInitLen/np.sum(SInitLen)
    dfCL.Init[catchment] = net_init







dfCL_org = dfCL.copy()

S = np.array([1,2,3,4,5])

dfCL = dfCL_org.copy()



# Flushing Transition Matrix
FlushTran = np.array([[1, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0]])

<ipython-input-5-de5edefbeb04>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfT.Tran[i] = np.around(expm(Q), decimals=3)
<ipython-input-5-de5edefbeb04>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfT.Init[i]=np.array([0, 0, 0, 0, 0])


Streaming output truncated to the last 5000 lines.
[[-3.94  3.94  0.    0.    0.  ]
 [ 0.   -2.93  2.93  0.    0.  ]
 [ 0.    0.   -3.26  3.26  0.  ]
 [ 0.    0.    0.   -3.68  3.68]
 [ 0.    0.    0.    0.    0.  ]]
[[0.019 0.132 0.201 0.202 0.445]
 [0.    0.053 0.133 0.18  0.633]
 [0.    0.    0.038 0.102 0.859]
 [0.    0.    0.    0.025 0.975]
 [0.    0.    0.    0.    1.   ]]
[[-4.2   4.2   0.    0.    0.  ]
 [ 0.   -3.03  3.03  0.    0.  ]
 [ 0.    0.   -3.42  3.42  0.  ]
 [ 0.    0.    0.   -4.29  4.29]
 [ 0.    0.    0.    0.    0.  ]]
[[0.015 0.12  0.188 0.178 0.499]
 [0.    0.048 0.121 0.149 0.681]
 [0.    0.    0.033 0.075 0.893]
 [0.    0.    0.    0.014 0.986]
 [0.    0.    0.    0.    1.   ]]
[[-3.96  3.96  0.    0.    0.  ]
 [ 0.   -2.94  2.94  0.    0.  ]
 [ 0.    0.   -3.27  3.27  0.  ]
 [ 0.    0.    0.   -3.71  3.71]
 [ 0.    0.    0.    0.    0.  ]]
[[0.019 0.131 0.201 0.201 0.449]
 [0.    0.053 0.132 0.178 0.636]
 [0.    0.    0.038 0.101 0.861]
 [0.    0.    0.    

<ipython-input-5-de5edefbeb04>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCL.Tran[catchment] = net_tran
<ipython-input-5-de5edefbeb04>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCL.Init[catchment] = net_init


In [6]:
dfT

,ASSET_ID,Catchment_Group,Len,Current,Degradation_Rate,lambda1,lambda2,lambda3,lambda4,Tran,Init,Next,Mean
0,G-WWM01,1,66.90,1,0.48,0.00,0.00,0.00,0.00,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0....","[1, 0, 0, 0, 0]",,
1,G-WWM02,1,38.93,1,0.04,0.23,0.18,0.17,0.01,"[[0.795, 0.187, 0.017, 0.001, 0.0], [0.0, 0.83...","[1, 0, 0, 0, 0]",,
2,G-WWM03,1,29.20,0,0.02,0.08,0.07,0.06,0.00,"[[0.923, 0.074, 0.003, 0.0, 0.0], [0.0, 0.932,...","[1, 0, 0, 0, 0]",,
3,G-WWM04,1,14.17,1,0.02,0.54,0.52,0.48,0.05,"[[0.583, 0.318, 0.084, 0.015, 0.0], [0.0, 0.59...","[1, 0, 0, 0, 0]",,
4,G-WWM05,1,101.43,0,0.14,0.01,0.01,0.00,0.00,"[[0.99, 0.01, 0.0, 0.0, 0.0], [0.0, 0.99, 0.01...","[1, 0, 0, 0, 0]",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,G-WWM02871,PS4SN_5,69.19,0,0.06,3.94,2.93,3.25,3.67,"[[0.019, 0.132, 0.202, 0.202, 0.444], [0.0, 0....","[1, 0, 0, 0, 0]",,
2588,G-WWM02872,PS4SN_5,116.15,0,0.08,0.18,0.13,0.20,0.02,"[[0.835, 0.154, 0.01, 0.001, 0.0], [0.0, 0.878...","[1, 0, 0, 0, 0]",,
2589,G-WWM02873,PS4SN_5,3.36,0,0.04,0.23,0.28,0.23,0.01,"[[0.795, 0.178, 0.025, 0.002, 0.0], [0.0, 0.75...","[1, 0, 0, 0, 0]",,
2590,G-WWM02874,PS4SN_5,81.04,0,0.07,0.33,0.26,0.40,0.06,"[[0.719, 0.246, 0.031, 0.004, 0.0], [0.0, 0.77...","[1, 0, 0, 0, 0]",,


In [19]:
dfCL

,Len,Tran,Init,Next,Mean,MeanLen,PPA,PPALen,TPR
Catchment_Group,,,,,,,,,
1,2753.40,"[[0.5635173826477938, 0.23746525583532777, 0.1...","[0.8694128371728146, 0.05144166687852518, 0.03...",,,,,,"[3.4057066948711516, 2.9195649321828205, 2.570..."
11,1155.32,"[[0.2060486319941871, 0.35782612817456266, 0.2...","[0.9134179275006059, 0.08496347332340821, 0.0,...",,,,,,"[4.33594413903185, 3.7665358415166885, 3.42030..."
12,5716.90,"[[0.06357516541016403, 0.16251595823154133, 0....","[0.811433429189144, 0.12964148527528807, 0.026...",,,,,,"[4.821123260916424, 4.249712095624736, 4.09467..."
13_25,3021.68,"[[0.1201386765784925, 0.23158155422491242, 0.1...","[0.8270249698012675, 0.07742127645491702, 0.07...",,,,,,"[4.670172963049356, 4.098931695395644, 3.86327..."
17,7359.18,"[[0.40418886081881983, 0.302058724508553, 0.15...","[1.0, 0.0, 0.0, 0.0, 0.0]",,,,,,"[3.759292471820906, 3.2266293871820055, 2.8696..."
18,1411.03,"[[0.4242527651449543, 0.24172302762296047, 0.1...","[0.8560322038823254, 0.14396779611767455, 0.0,...",,,,,,"[3.8643976854153457, 3.3260342231246756, 2.967..."
19,1146.82,"[[0.3941009182024427, 0.2726488009268558, 0.17...","[0.8392089498879521, 0.05136768309165264, 0.0,...",,,,,,"[3.9567324328002624, 3.41395656889182, 3.05147..."


In [15]:


def getCatchmentPlan(plan):
    """
    Converts the entire network plan into a dictionary with a separate plan for each catchment
    :param schedule: a list of binary values describing the given schedule
    :return: a dictionary with each catchment as a key and the corresponding plan as the value
    """
    
    catchmentplanDict = {}
    shiftIndex = 0

    for catchment in catchments:
        catchmentplanDict[catchment] = plan[shiftIndex:shiftIndex + num_years]
        shiftIndex += num_years

    return catchmentplanDict

def condition(CatchmentPlan,catchment):
    """
    Calculate the states of each catchment throughout the period according to the plan
    :param CatchmentPlan: a list of binary values describing the catchment plan
    :return: a list of numpy arrays presenting the states in each year
    """
    
    state_list = []
    state = dfCL.Init[catchment]
    for i in range(len(CatchmentPlan)):
        new_state = (1-CatchmentPlan[i])*np.dot(state,dfCL.Tran[catchment])+ CatchmentPlan[i]*np.dot(state,FlushTran)
        state_list.append(new_state)
        state = new_state.copy()
    return state_list

def nextcond():
    dfCL.Next = dfCL.apply(lambda x: np.dot(x.Init, x.Tran), axis=1)
    dfCL.Mean = dfCL.apply(lambda x: (np.dot(S, x.Next.T)), axis=1)
    dfCL.MeanLen = dfCL.Mean*dfCL.Len
    dfCL.PPA = dfCL.apply(lambda x: x.Next[0,3]+x.Next[0,4], axis=1)
    dfCL.PPALen = dfCL.PPA*dfCL.Len
    


def getObjCatchment(CatchmentStates):
    MeanC_Catchment = []
    PPA_Catchment = []
    for i in range(len(CatchmentStates)):
        MeanC_Catchment.append(np.dot(S,CatchmentStates[i].T))
        PPA_Catchment.append(CatchmentStates[i][3]+CatchmentStates[i][4])

    return sum(MeanC_Catchment)/num_years , sum(PPA_Catchment)/num_years , MeanC_Catchment , PPA_Catchment


def getObjCatchment2(CatchmentPlan,catchment):
    dfC = dfT.loc[dfT['Catchment_Group'] == catchment].copy()
    condition_list=[]
    for i in range(len(CatchmentPlan)):
        if CatchmentPlan[i]==1:
            dfC.Next = dfC.apply(lambda x: np.dot(x.Init, FlushTran), axis=1)
        else:
            dfC.Next = dfC.apply(lambda x: np.dot(x.Init, x.Tran), axis=1)
        dfC.Mean = dfC.apply(lambda x: sum(sum((np.dot(S, x.Next.T)))), axis=1)
        obj = sum(dfC.Len*dfC.Mean)/sum(dfC.Len)
        condition_list.append(obj)
        dfC.Init = dfC.Next.copy()
    return sum(condition_list)/len(condition_list)


def BudgetConstraint(catchmentplanDict):
    num_hard_violation = 0
    percent = 0
    for i in range(num_years):
        cost = 0
        for C in catchments:
            cost += catchmentplanDict[C][i]*dfCL.Len[C]*Ucost
        if cost > budgetb*(1+Tol):
            num_hard_violation +=1
            percent += abs(cost-budgetb*(1+Tol))/(budgetb*(1+Tol))
        if cost < budgetb*(1-Tol):
            num_hard_violation +=1
            percent += abs(cost-budgetb*(1-Tol))/(budgetb*(1-Tol))
    
    return num_hard_violation, percent

def BudgetConstraintTotal(catchmentplanDict):
    num_budget_violation = 0
    total_budget_percent = 0
    costT = 0
    budgetTotal = budgetb*num_years
    for i in range(num_years):
        cost = 0
        for C in catchments:
            cost += catchmentplanDict[C][i]*dfCL.Len[C]*Ucost
        costT += cost
    if costT > budgetTotal:
        num_budget_violation +=1
        total_budget_percent += (costT-budgetTotal)/budgetTotal

    
    return num_budget_violation, total_budget_percent

def OneFlushConstraint(catchmentplanDict):
    num_soft_violation = 0
    soft = 0
    for C in catchments:
        if sum(catchmentplanDict[C]) == 0:
            num_soft_violation +=1
            soft += dfCL.Len[C]
    soft_percent = soft/dfCL.Len.sum()       
    return num_soft_violation, soft_percent  



def getObj(plan):
    """
    Calculates the total cost of the various violations in the given network plan
    ...
    :param plan: a list of binary values describing the given network plan
    :return: the calculated cost
    """
    # convert entire schedule into a dictionary with a separate schedule for each catchment:
    catchmentplanDict = getCatchmentPlan(plan)
    MeanC_Catchment_List = []
    PPA_Catchment_List = []
    for C in catchments:
        CatchmentStates = condition(catchmentplanDict[C],C)
        MeanC_Catchment , PPA_Catchment , _ , _  = getObjCatchment(CatchmentStates)
        MeanC_Catchment_List.append(MeanC_Catchment) 
        PPA_Catchment_List.append(PPA_Catchment)
    #print(Obj_Catchment_List)
    if Criteria == "PPA":
        Obj_Network = (sum(np.array(dfCL.Len)*np.array(PPA_Catchment_List)))/sum(dfCL.Len)
    else:
        Obj_Network = (sum(np.array(dfCL.Len)*np.array(MeanC_Catchment_List)))/sum(dfCL.Len)
    
    
    #add penalty for violating constraint
    num_hard_violation , percent_hard_violation = BudgetConstraint(catchmentplanDict)
    num_soft_violation , percent_soft_violation = OneFlushConstraint(catchmentplanDict)
    num_budget_violation, total_budget_percent = BudgetConstraintTotal(catchmentplanDict)
    
    #Obj_Network_WC = Obj_Network+num_hard_violation*HARD_CONSTRAINT_PENALTY+num_soft_violation*SOFT_CONSTRAINT_PENALTY
    Obj_Network_WC = Obj_Network+percent_hard_violation*HARD_CONSTRAINT_PENALTY+percent_soft_violation*SOFT_CONSTRAINT_PENALTY+BUDGET_TOTAL_PENALTY*total_budget_percent
    return Obj_Network_WC , Obj_Network

def getPerformance(plan):
    """
    Calculates the total cost of the various violations in the given network plan
    ...
    :param plan: a list of binary values describing the given network plan
    :return: the calculated cost
    """
    # convert entire schedule into a dictionary with a separate schedule for each catchment:
    
    performance = pd.DataFrame(columns = [i for i in range(1,num_years+1)], index = catchments)
    
    catchmentplanDict = getCatchmentPlan(plan)
    for C in catchments:
        CatchmentStates = condition(catchmentplanDict[C],C)
        _, _, MeanC_Catchment , PPA_Catchment = getObjCatchment(CatchmentStates)
        if Criteria == "PPA":
            performance.loc[C] = [i*dfCL.Len[C]/dfCL.Len.sum() for i in PPA_Catchment]
        else:
            performance.loc[C] = [i*dfCL.Len[C]/dfCL.Len.sum() for i in MeanC_Catchment]
    return performance.sum().tolist()    


def printPlanInfo(plan):
    """
    Prints the plan and violations details
    :param schedule: a list of binary values describing the given plan
    """
    catchmentplanDict = getCatchmentPlan(plan)

    print("Plan for each Catchment:")
    for catchment in catchmentplanDict: 
        print(catchment, ":", catchmentplanDict[catchment])
    print()
    print("Yearly Budget violations = ", BudgetConstraint(catchmentplanDict))
    print()
    print("Total Budget violations = ", BudgetConstraintTotal(catchmentplanDict))
    print()
    print("Minimum flush violations = ", OneFlushConstraint(catchmentplanDict))
    print()
    
    required_budget = []
    for i in range(num_years):
        cost = 0
        for C in catchments:
            cost += catchmentplanDict[C][i]*dfCL.Len[C]*Ucost
        required_budget.append(cost)
    print("Required budget for this plan: ", [round(num, 1) for num in required_budget] )
    print()
    print("Total required budget: ", round(sum(required_budget), 1))
    print()
    Obj_Network_WC , Obj_Network = getObj(plan)
    print("Network objective without penalty: ", Obj_Network )
    print()
    print(pd.DataFrame(np.array(plan).reshape(num_catchments,num_years),index=catchments, columns=range(1,1+num_years)))
    



def Optimizeplan():  
    plan = pd.DataFrame(index = catchments)
    for year in range(1,num_years):
        plan[str(year)]=''
        nextcond()
        model = LpProblem(name="Flushing-Plan", sense=LpMinimize)
        x = {i: LpVariable(name="{0}".format(dfCL.index[i]), lowBound=0, upBound=1, cat=LpInteger ) for i in range(len(catchments))}
        if Criteria == "PPA":        
            model += sum(dfCL.PPALen)-lpDot(dfCL.PPALen.tolist(), x.values())
        else:
            model += sum(dfCL.MeanLen)-lpDot(dfCL.MeanLen.tolist(), x.values())+lpDot(dfCL.Len.tolist(), x.values())
            
        model += (lpDot(dfCL.Len.tolist(), x.values())*Ucost <= budget[year-1] , "Budget Constraint")
        status = model.solve()
        for var in x.values():
            #print(f"{var.name}: {var.value()}")
            if var.value()==1:
                plan[str(year)][var.name]=1
            else:
                plan[str(year)][var.name]=0

        for c in catchments:
            if plan[str(year)][c]==1:
                dfCL['Init'][c] = np.dot(dfCL['Next'][c], FlushTran)
            else:
                dfCL['Init'][c] = dfCL['Next'][c]

    year = num_years
    noflush = []
    noflushcost = []
    for c in catchments:
        if plan.loc[c,:].sum() == 0:
            noflush.append(c)
            noflushcost.append(dfCL.Len[c]*Ucost) 
    if sum(noflushcost) > budget[year-1]:
        print()
        print()
        print('The budget is not enough for the remaining catchments')
        print()
        print()

    plan[str(year)]=''
    nextcond()
    model = LpProblem(name="Flushing-Plan", sense=LpMinimize)
    x = {i: LpVariable(name="{0}".format(dfCL.index[i]), lowBound=0, upBound=1, cat=LpInteger ) for i in range(len(catchments))}
    if Criteria == "PPA":        
        model += sum(dfCL.PPALen)-lpDot(dfCL.PPALen.tolist(), x.values())
    else:
        model += sum(dfCL.MeanLen)-lpDot(dfCL.MeanLen.tolist(), x.values())+lpDot(dfCL.Len.tolist(), x.values())
    model += (lpDot(dfCL.Len.tolist(), x.values())*Ucost <= budget[year-1] , "Budget Constraint")
    for c in noflush:
        catchmentlist = catchments.values.tolist()
        model += (x[catchmentlist.index(c)] == 1)
    status = model.solve()
    for var in x.values():
        #print(f"{var.name}: {var.value()}")
        if var.value()==1:
            plan[str(year)][var.name]=1
        else:
            plan[str(year)][var.name]=0
    
    return planlist(plan)

def planlist(plan):
    sol = []
    for c in catchments:
        for year in range(1,num_years+1):
            sol.append(plan[str(year)][c])
    return sol




Tplan = []
for i in range(2**num_years):
    strplan = (format(i, '0'+str(num_years)+'b'))
    plan = [int(j) for j in strplan]
    Tplan.append(plan)
    


for C in tqdm(catchments):
    TPR = []
    for p in Tplan:
        dfCL = dfCL_org.copy()
        CatchmentStates = condition(p,C)
        MeanC_Catchment , _ , _ , _  = getObjCatchment(CatchmentStates)
        TPR.append(MeanC_Catchment)
    dfCL_org.TPR[C] = np.array(TPR)

    
Cost = {}
for x in range(1, num_years+1):
    y = []
    for p in Tplan:
        y.append(p[x-1])
    Cost["{0}".format(x)] = y
    
    
def PlanInfo(plan):
    """
    Prints the plan and violations details
    :param schedule: a list of binary values describing the given plan
    """
    catchmentplanDict = getCatchmentPlan(plan)
    
    required_budget = []
    for i in range(num_years):
        cost = 0
        for C in catchments:
            cost += catchmentplanDict[C][i]*dfCL.Len[C]*Ucost
        required_budget.append(cost)
        
    _ , Obj_Network = getObj(plan)
    return Obj_Network , round(sum(required_budget), 3)

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-15-233643dcfafc>:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCL_org.TPR[C] = np.array(TPR)
100%|██████████| 7/7 [00:00<00:00, 24.20it/s]


In [8]:

# Genetic Algorithm constants:
POPULATION_SIZE = 300
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.1   # probability for mutating an individual
MAX_GENERATIONS = 100
HALL_OF_FAME_SIZE = 100
HARD_CONSTRAINT_PENALTY = 30
BUDGET_TOTAL_PENALTY = 30
SOFT_CONSTRAINT_PENALTY = 100



In [9]:
import itertools
alist = []
falist = []
fcilist = []
costlist = []
fcostlist = []
ccost = np.asarray(dfCL.Len)*Ucost
# Define problem state (list of nonnegative integers)
action_space = [1]*num_catchments

# Create list of sets based on state
list_of_sets=[(j for j in range(action_space[i]+1)) for i in range(len(action_space))]

# Compute Cartesian product
num_actions=0
for action in itertools.product(*list_of_sets):
    print(action)
    num_actions +=1
    alist.append(action)
    action_cost = sum(ccost*np.asarray(action))
    costlist.append(action_cost)
    if (action_cost <= budgetb*(1+Tol) and action_cost >= budgetb*(1-Tol)):
        fci = 1
        falist.append(action)
        fcostlist.append(round(action_cost,1))
    else:
        fci = 0
    fcilist.append(fci)
    
    
print('Number of actions generated:', num_actions)
print('Number of feasible actions:', sum(fcilist))

adf = pd.DataFrame({'action':falist,'cost':fcostlist})

Tadf = pd.DataFrame({'action':alist,'cost':costlist})


(0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 1, 0, 1)
(0, 0, 0, 0, 1, 1, 0)
(0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 1, 0, 0, 1)
(0, 0, 0, 1, 0, 1, 0)
(0, 0, 0, 1, 0, 1, 1)
(0, 0, 0, 1, 1, 0, 0)
(0, 0, 0, 1, 1, 0, 1)
(0, 0, 0, 1, 1, 1, 0)
(0, 0, 0, 1, 1, 1, 1)
(0, 0, 1, 0, 0, 0, 0)
(0, 0, 1, 0, 0, 0, 1)
(0, 0, 1, 0, 0, 1, 0)
(0, 0, 1, 0, 0, 1, 1)
(0, 0, 1, 0, 1, 0, 0)
(0, 0, 1, 0, 1, 0, 1)
(0, 0, 1, 0, 1, 1, 0)
(0, 0, 1, 0, 1, 1, 1)
(0, 0, 1, 1, 0, 0, 0)
(0, 0, 1, 1, 0, 0, 1)
(0, 0, 1, 1, 0, 1, 0)
(0, 0, 1, 1, 0, 1, 1)
(0, 0, 1, 1, 1, 0, 0)
(0, 0, 1, 1, 1, 0, 1)
(0, 0, 1, 1, 1, 1, 0)
(0, 0, 1, 1, 1, 1, 1)
(0, 1, 0, 0, 0, 0, 0)
(0, 1, 0, 0, 0, 0, 1)
(0, 1, 0, 0, 0, 1, 0)
(0, 1, 0, 0, 0, 1, 1)
(0, 1, 0, 0, 1, 0, 0)
(0, 1, 0, 0, 1, 0, 1)
(0, 1, 0, 0, 1, 1, 0)
(0, 1, 0, 0, 1, 1, 1)
(0, 1, 0, 1, 0, 0, 0)
(0, 1, 0, 1, 0, 0, 1)
(0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 0, 1, 1)
(0, 1, 0, 1, 1, 0, 0)
(0, 1, 0, 

In [11]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict
import random

import numpy as np
import matplotlib.pyplot as plt

In [10]:
Tadf

,action,cost
0,"(0, 0, 0, 0, 0, 0, 0)",0.0000
1,"(0, 0, 0, 0, 0, 0, 1)",1582.6116
2,"(0, 0, 0, 0, 0, 1, 0)",1947.2214
3,"(0, 0, 0, 0, 0, 1, 1)",3529.8330
4,"(0, 0, 0, 0, 1, 0, 0)",10155.6684
...,...,...
123,"(1, 1, 1, 1, 0, 1, 1)",20983.1070
124,"(1, 1, 1, 1, 1, 0, 0)",27608.9424
125,"(1, 1, 1, 1, 1, 0, 1)",29191.5540
126,"(1, 1, 1, 1, 1, 1, 0)",29556.1638


In [13]:
trans = np.zeros(shape=[num_catchments,2,5,5])
trans[:, 1, :, 0] = 1.0  # replace action
# no-actions for all components
for i in range (num_catchments):
    trans[i,0] = dfCL.Tran[i]

num_years = 7
Total_budget = num_years*budgetb
BPF = 50  #budget penalty factor

actiondf = Tadf                                           ################### Action List ##############################################################

class SewerEnv(Env):
    def __init__(self):
        # Actions for 7 catchments
        self.action_space = Discrete(len(actiondf))
        self.time = 0
        self.components = num_catchments
        self.observation_space = Box(low=0.0, high=1.0, shape=(num_catchments+3, ), dtype=np.float32)
        self.state = self.reset()

    
    def reset(self):
        self.time = 0.0
        self.cond = np.stack(dfCL.Init.values)
        avg_cond = np.matmul(self.cond, S)
        s_cond = np.dot(avg_cond,dfCL.Len)/dfCL.Len.sum()
        self.RB = 1.0
        self.state = self.observation_space.sample()
        self.state[:num_catchments] = (avg_cond-1)/4
        self.state[num_catchments] = (s_cond-1)/4
        self.state[num_catchments+1] = self.time/num_years
        self.state[num_catchments+2] = self.RB

        return self.state
    
    
    

    def step(self, action, render=False):
                
        # Return step information
        self.time += 1
        done = False
        if self.time >= num_years:
            done = True

        pcost = actiondf.loc[action].values[1]
        self.RB -= pcost/Total_budget 
        action = actiondf.loc[action].values[0]
        # transition possibility matrices
            
        # deterioration
        for component in range(self.components):  # for each component
            self.cond[component] = np.matmul(self.cond[component],trans[component, action[component]])

 
        avg_cond = np.matmul(self.cond, S) + np.random.normal(0,0.001,num_catchments)
        s_cond = np.dot(avg_cond,dfCL.Len)/dfCL.Len.sum()
        self.state = self.observation_space.sample()
        self.state[:num_catchments] = (avg_cond-1)/4
        self.state[num_catchments] = (s_cond-1)/4
        self.state[num_catchments+1] = self.time/num_years
        self.state[num_catchments+2] = self.RB
        
        # calculate reward
        reward = 1 - self.state[num_catchments]
        
        # if done:
  
        if (self.RB < (1-self.state[num_catchments+1])*(1-Tol)):
          # done = True
          reward -= num_years/14
  
        if pcost > budgetb*(1+Tol):
          reward -= 1
          # reward -= BPF*(pcost-budgetb*(1+Tol))/(budgetb)
          # done = True
        if pcost < budgetb*(1-Tol):
          reward -= 1
          # reward -= BPF*(budgetb*(1-Tol)-pcost)/(budgetb)        
          # done = True
        
        info = {}
        
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    

    

In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import gym

class prioritized_replay_buffer(object):
    def __init__(self, capacity, alpha, beta, beta_increment):
        self.capacity = capacity
        self.alpha = alpha
        self.beta = beta
        self.beta_increment = beta_increment
        self.pos = 0
        self.memory = []
        self.priorities = np.zeros([self.capacity], dtype=np.float32)

    def store(self, observation, action, reward, next_observation, done):
        observation = np.expand_dims(observation, 0)
        next_observation = np.expand_dims(next_observation, 0)

        max_prior = np.max(self.priorities) if self.memory else 1.0

        if len(self.memory) < self.capacity:
            self.memory.append([observation, action, reward, next_observation, done])
        else:
            self.memory[self.pos] = [observation, action, reward, next_observation, done]
        self.priorities[self.pos] = max_prior
        self.pos += 1
        self.pos = self.pos % self.capacity

    def sample(self, batch_size):
        if len(self.memory) < self.capacity:
            probs = self.priorities[: len(self.memory)]
        else:
            probs = self.priorities
        probs = probs ** self.alpha
        probs = probs / np.sum(probs)

        indices = np.random.choice(len(self.memory), batch_size, p=probs)
        samples = [self.memory[idx] for idx in indices]

        weights = (len(self.memory) * probs[indices]) ** (- self.beta)
        if self.beta < 1:
            self.beta += self.beta_increment
        weights = weights / np.max(weights)
        weights = np.array(weights, dtype=np.float32)

        observation, action, reward, next_observation, done = zip(* samples)
        return np.concatenate(observation, 0), action, reward, np.concatenate(next_observation, 0), done, indices, weights

    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority

    def __len__(self):
        return len(self.memory)


class dqn(nn.Module):
    def __init__(self, observation_dim, action_dim):
        super(dqn, self).__init__()
        self.observation_dim = observation_dim
        self.action_dim = action_dim

        self.fc1 = nn.Linear(self.observation_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, self.action_dim)

    def forward(self, observation):
        x = self.fc1(observation)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

    def get_action(self, observation, epsilon):
        if random.random() > epsilon:
            q_value = self.forward(observation)
            action = q_value.max(1)[1].data[0].item()
        else:
            rtime = observation[0][num_catchments+1].numpy().item()
            rbudget = observation[0][num_catchments+2].numpy().item()
            tbudget = (rbudget-(1-(rtime+1/num_years))*(1-Tol))*Total_budget
            factions = Tadf[(Tadf.cost >= budgetb*(1-Tol)) & (Tadf.cost <= budgetb*(1+Tol)) & (Tadf.cost <= tbudget)]
            if len(factions) != 0:
              action = random.choice(factions.index)
            else:
              q_value = self.forward(observation)
              action = q_value.max(1)[1].data[0].item()
        return action


def training(buffer, batch_size, model, optimizer, gamma, loss_fn):
    observation, action, reward, next_observation, done, indices, weights= buffer.sample(batch_size)

    observation = torch.FloatTensor(observation)
    action = torch.LongTensor(action)
    reward = torch.FloatTensor(reward)
    next_observation = torch.FloatTensor(next_observation)
    done = torch.FloatTensor(done)
    weights = torch.FloatTensor(weights)

    q_values = model.forward(observation)
    next_q_values = model.forward(next_observation)

    q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value = next_q_values.max(1)[0].detach()
    expected_q_value = reward + next_q_value * (1 - done) * gamma

    loss = (expected_q_value - q_value).pow(2) * weights
    priorities = loss + 1e-5
    buffer.update_priorities(indices, priorities)
    loss = loss.mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()





In [16]:
training_reward_list = []
from collections import deque
scores_window = deque(maxlen=100)  # last 100 scores
if __name__ == '__main__':
    t0 = time.time()
    epsilon_init = 1.0
    epsilon_min = 0.001
    episode = 5000*4
    decay = 0.9995
    capacity = int(num_years*episode/5)
    exploration = int(episode/100)
    batch_size = 64 
    render = False
    learning_rate = 1e-4
    gamma = 0.99
    alpha = 0.6
    beta = 0.4
    beta_increment_step = int(episode/2)
    loss_fn = nn.MSELoss()

    eps_frames = episode/2
    eps_start = 1.0
    min_eps = 0.05

    # env = gym.make('CartPole-v0')
    env = SewerEnv()
    env = env.unwrapped
    action_dim = env.action_space.n
    observation_dim = env.observation_space.shape[0]

    model = dqn(observation_dim, action_dim)
    beta_increment = (1 - beta) / beta_increment_step
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    buffer = prioritized_replay_buffer(capacity, alpha, beta, beta_increment=beta_increment)
    epsilon = epsilon_init
    weight_reward = True

    for i in range(episode):
        obs = env.reset()
        if (epsilon > epsilon_min) and (i > exploration):
            epsilon = epsilon * decay

        # if i < eps_frames:
        #     epsilon = max(eps_start - (i*(1/eps_frames)), min_eps)
        # else:
        #     epsilon = max(min_eps - min_eps*((i-eps_frames)/(episode-eps_frames)), 0.001)
        
        reward_total = 0
        if render:
            env.render()
        while True:           
            action = model.get_action(torch.FloatTensor(np.expand_dims(obs, 0)), epsilon)
            train_flag = False
            next_obs, reward, done, info = env.step(action)
            if render:
                env.render()
            buffer.store(obs, action, reward, next_obs, done)
            reward_total += reward
            obs = next_obs
            if len(buffer) > exploration:
                training(buffer, batch_size, model, optimizer, gamma, loss_fn)
                train_flag = True
            if done:
                scores_window.append(reward_total)
                training_reward_list.append(reward_total)
                if not weight_reward:
                    weight_reward = reward_total
                else:
                    weight_reward = 0.99 * weight_reward + 0.01 * reward_total
                print('\repisode: {}  reward: {}  epsilon: {:.2f}  train:  {}  weight_reward: {:.3f}'.format(i+1, reward_total, epsilon, train_flag, weight_reward), end="")
                if i % 1000 == 0:
                   print('\repisode: {}  reward: {}  epsilon: {:.2f}  train:  {}  weight_reward: {:.3f}'.format(i+1, reward_total, epsilon, train_flag, weight_reward))
                break
      
    t1 = time.time() 
    print("Training time: {}min".format(round((t1-t0)/60,2)))

episode: 1  reward: 4.128547891974449  epsilon: 1.00  train:  False  weight_reward: 1.031
episode: 1001  reward: 3.9214044362306595  epsilon: 0.67  train:  True  weight_reward: 3.924
episode: 2001  reward: 4.295092761516571  epsilon: 0.41  train:  True  weight_reward: 4.097
episode: 3001  reward: 4.266873747110367  epsilon: 0.25  train:  True  weight_reward: 4.270
episode: 4001  reward: 4.34587587416172  epsilon: 0.15  train:  True  weight_reward: 4.320
episode: 5001  reward: 4.373915940523148  epsilon: 0.09  train:  True  weight_reward: 4.395
episode: 6001  reward: 4.459305241703987  epsilon: 0.05  train:  True  weight_reward: 4.403
episode: 7001  reward: 4.435511112213135  epsilon: 0.03  train:  True  weight_reward: 4.430
episode: 8001  reward: 4.47058467566967  epsilon: 0.02  train:  True  weight_reward: 4.437
episode: 9001  reward: 4.527149438858032  epsilon: 0.01  train:  True  weight_reward: 4.431
episode: 10001  reward: 4.453086942434311  epsilon: 0.01  train:  True  weight_rewa

In [17]:
reward_total = 0
eps = 0
obs = env.reset()
done = False
reward_list = []
action_list = []
while not done:
    action = model.get_action(torch.FloatTensor(np.expand_dims(obs, 0)), eps)
    print(action)
    action_list.append(action)
    next_obs, reward, done, info = env.step(action)
    buffer.store(obs, action, reward, next_obs, done)
    reward_total += reward
    obs = next_obs
print(reward_total)
((1-(reward_total/num_years))*4+1)

48
73
48
4
18
105
18
4.5098636001348495


2.4229350856372287

In [ ]:
sol = np.array([])
for acode in action_list:
    sol = np.append(sol,np.array(actiondf.loc[acode].values[0]))
sol = sol.reshape(num_years,num_catchments).T.flatten().tolist()
print(sol)
printPlanInfo(sol)

[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Plan for each Catchment:
1 : [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
11 : [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
12 : [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
13_25 : [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
17 : [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
18 : [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
19 : [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

Yearly Budget violations =  (0, 0)

Total Budget violations =  (0, 0)

Minimum flush violations =  (0, 0.0)

Required budget for this plan:  [9471.9, 9916.8, 9483.7, 10155.7, 9836.5, 11146.6, 9483.7]

Total required budget:  69494.9

Network objective without penalty:  2.4195280081776325

                   1    2    3    4    5    6    7
Catchment_Group                                   
1                0.0  1.0  0.0  0.0  0.0  1.0  

In [21]:
scores_window = deque(maxlen=100) 
average_100 = []

In [22]:
for i in range(len(training_reward_list)):
  scores_window.append(training_reward_list[i])
  average_100.append(np.mean(scores_window))


In [23]:
fig = plt.figure(figsize=(12,7))
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(training_reward_list)+1), training_reward_list, alpha=0.6, label = " Training rewards")
plt.plot(np.arange(1, len(average_100)+1), average_100, label = "100 average rewards")
plt.legend()
plt.ylabel('Rewards')
plt.xlabel('Episode #')
plt.show()